In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ

In [3]:
feedback_bits = 64
ncodebooks=512
ncentroids=128
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test_f%i.npy' % feedback_bits
    featurepath_test = 'feature_test_f%i.npy' % feedback_bits
    y_test = 'y_test_f%i.npy' % feedback_bits
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw_f%i.npy' % feedback_bits
biaspath = 'encoder_fcb_f%i.npy' % feedback_bits
dir_result = os.path.join(dir_result, method)


In [6]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral
learn_multisplits(): initial loss:    13.817902764871128
learn_multisplits(): returning loss:  6.205424594372744
learn_multisplits(): initial loss:    110.96574758379359
learn_multisplits(): returning loss:  36.7686811129115
learn_multisplits(): initial loss:    10.540118146729256
learn_multisplits(): returning loss:  5.045186639763415
learn_multisplits(): initial loss:    42.459335356375504
learn_multisplits(): returning loss:  7.5348684075288475
learn_multisplits(): initial loss:    10.537402315256433
learn_multisplits(): returning loss:  4.9522933364787605
learn_multisplits(): initial loss:    12.607044605409339
learn_multisplits(): returning loss:  7.011695434657668
learn_multisplits(): initial loss:    8.754820130879668
learn_multisplits(): returning loss:  4.946853552013636
learn_multisplits(): initial loss:    9.072896923857732
learn_multisplits(): returning loss:  4.688595763873309
learn_multisplits(): initial loss:    61.34939141141249
learn_multisplit

/data/hdr/pq/bolt/experiments/python/clusterize.py:1436: UserWarning: Persisting input arguments took 0.80s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, pq_perm_algo='start')


X_res mse / X mse:  0.34515205
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.12240425


/data/hdr/pq/bolt/experiments/python/vquantizers.py:610: UserWarning: Persisting input arguments took 0.80s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.splits_lists, self.centroids = clusterize.learn_mithral(


In [7]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 0.20093924  0.03357202  0.2990297  ...  0.01819205 -0.06221606
  -0.2748678 ]
 [ 0.20093924  0.7850397  -0.13932645 ...  0.5191705  -0.3127053
  -0.46273473]
 [ 0.01307232  0.09619433  0.73738587 ...  1.020149   -0.12483837
   0.35135528]
 ...
 [ 0.20093924 -0.02905029 -0.20194876 ...  0.83228207  1.378097
  -0.5879794 ]
 [-0.73839545  0.09619433  0.11116278 ... -0.2322972   0.18827316
  -0.21224551]
 [ 0.7645401   0.22143894 -0.13932645 ... -0.795898   -0.37532762
   0.22611065]]
(32000, 64)
